In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [33]:

# ========== Load Data ==========
def load_chunks(file_path, label, chunk_size=256, max_chunks=390):
    with open(file_path, 'r') as f:
        data = f.read().replace('\n', '').strip()
    data = [int(b) for b in data]
    chunks = []
    labels = []
    for i in range(0, len(data) - chunk_size + 1, chunk_size):
        chunks.append(data[i:i + chunk_size])
        labels.append(label)
        if len(chunks) >= max_chunks:
            break
    return np.array(chunks, dtype=np.float32), np.array(labels, dtype=np.float32)

print("Loading data...")
X_qrng, y_qrng = load_chunks('qrng_1.txt', 1.0)
X_prng, y_prng = load_chunks('prng_2.txt', 0.0)
print("QRNG shape:", X_qrng.shape, "Label:", y_qrng.shape)
print("PRNG shape:", X_prng.shape, "Label:", y_prng.shape)
print("Total X shape:", X.shape, "| y shape:", y.shape)
print("Label distribution:", np.unique(y, return_counts=True))

X = np.concatenate([X_qrng, X_prng])
y = np.concatenate([y_qrng, y_prng])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True)


Loading data...
QRNG shape: (390, 256) Label: (390,)
PRNG shape: (390, 256) Label: (390,)
Total X shape: (780, 256) | y shape: (780,)
Label distribution: (array([0., 1.], dtype=float32), array([390, 390], dtype=int64))


In [34]:
train_ds = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
val_ds = TensorDataset(torch.tensor(X_val), torch.tensor(y_val))

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)

# ========== Define ANN Model ==========
class DeepEntropyANN(nn.Module):
    def __init__(self, input_size=256):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [35]:
# ========== Training Setup ==========
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = DeepEntropyANN().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# ========== Train and Evaluate ==========
def train(model, loader):
    model.train()
    total_loss = 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device).unsqueeze(1)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device).unsqueeze(1)
            preds = model(xb)
            preds = (preds > 0.5).float()
            correct += (preds == yb).sum().item()
            total += yb.size(0)
    return correct / total


In [37]:
# ========== Run Training ==========
print("Training...")
for epoch in range(10):
    loss = train(model, train_loader)
    acc = evaluate(model, val_loader)
    print(f"Epoch {epoch+1} | Loss: {loss:.4f} | Accuracy: {acc:.4f}")

Training...
Epoch 1 | Loss: 0.5447 | Accuracy: 0.5064
Epoch 2 | Loss: 0.4959 | Accuracy: 0.5064
Epoch 3 | Loss: 0.4391 | Accuracy: 0.5064
Epoch 4 | Loss: 0.3948 | Accuracy: 0.5321
Epoch 5 | Loss: 0.3492 | Accuracy: 0.5385
Epoch 6 | Loss: 0.3075 | Accuracy: 0.4808
Epoch 7 | Loss: 0.2407 | Accuracy: 0.5192
Epoch 8 | Loss: 0.2029 | Accuracy: 0.5192
Epoch 9 | Loss: 0.1504 | Accuracy: 0.5256
Epoch 10 | Loss: 0.1210 | Accuracy: 0.5064


In [40]:
def load_binary_string(file_path, chunk_size=256):
    with open(file_path, 'r') as f:
        data = f.read().replace('\n', '').strip()
    data = [int(b) for b in data[:chunk_size]]  # use only 1000 bits
    return torch.tensor(data, dtype=torch.float32).unsqueeze(0).to(device)

In [41]:
# Load your trained model (if not already in memory)
model.eval()  # put model in evaluation mode

# Load the test string from file
x_test = load_binary_string('prng_1.txt')  # new file here

# Get prediction
with torch.no_grad():
    pred = model(x_test).item()

print(f"Prediction: {pred:.4f} (closer to 1 = QRNG, closer to 0 = PRNG)")


Prediction: 0.6316 (closer to 1 = QRNG, closer to 0 = PRNG)
